In [ ]:
# The code was removed by Watson Studio for sharing.

# The Battle of Neighborhood - Part2

### Introduction & Bussiness justification

#### Background

The City of New York, is the most populous city in the United States. It is the financial capital of USA. This also means that the market is highly competitive. It provides lot of business oppourtunities and business friendly environment. It has attracted many different players into the market. It is a global hub of business and commerce. There are more than 2 million small businesses in New York state, which employ 4 million people. These small businesses make up 99.8% of all businesses within the state and employ more than half the state's workforce.

 People from diverse area lives here. As it is highly developed city so cost of doing business is also one of the highest. With it’s diverse culture , comes diverse food items. There are many restaurants in New York City, each belonging to different categories like Chinese , Indian , French etc. We need to analyze which immigrants are more, so that we can get an idea about the resturant style.

#### Things to keep in mind: 

A restaurant is such a business where the business personel has to keep in mind customer demand based on the population type nearby. The popular cuisine nearby. The City of New York & Toronto is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.

Central and Eastern European immigrants, especially Jewish immigrants - bagels, cheesecake, hot dogs etc
Italian immigrants -  pizza and Italian cuisine
Chinese , Asian and Middle Eastern restaurants- sandwich joints, lebanese, kababs and many more
mobile food vendors - Some 4,000 licensed by the city.

So it is evident that to survive in such competitive market it is very important to startegically plan. Various factors need to be studied inorder to decide on the Location such as :

New York Population including immigrants & natives.
Markets, Wholesale markets etc nearby so that the ingredients can be purchased fresh to maintain quality and cost.
venues like Gyms, Entertainmnet zones, Parks etc nearby where floating population is high etc
already established branded open resturants & food joints.
Varieties available there.
Segmentation of the Borough
Saturated markets etc
The list can go on...
Eventhough well funded,The Company need to choose the correct location to start its first venture.If this is successful they can plan about expantion, but the first foot step is needs to be very calculative.

#### Target Audience:

To recommend the correct location,Company has appointed the Data Science team. The objective is to locate and recommend to the management which neighborhood of Newyork city will be best choice to start a restaurant. The Management also expects the team to keep their demands in mind.
This would be a challenging job now to hit the correct audience.

#### Challenges to face:

We need to compare two cities with their geographical boundaries. We need to compare different data sets. We need to provide detailed analysis on different datasets.

### Prerequisites 

#### Data

1. NewYork DataSet :  https://geo.nyu.edu/catalog/nyu_2451_34572
2. Toronto DataSet : https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a
3. Toronto Neighborhoods : https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#a45bd45a-ede8-730e-1abc-93105b2c439f
4. New York Markets : https://data.cityofnewyork.us/dataset/DOHMH-Farmers-Markets-and-Food-Boxes/8vwk-6iz2
5. NewYork cuisine : https://en.wikipedia.org/wiki/Cuisine_of_New_York_City
6. Toronto cuisine : https://en.wikipedia.org/wiki/Cuisine_in_Toronto
7. Toronto Markets : https://en.wikipedia.org/wiki/St._Lawrence_Market

In [1]:

import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

#### Importing Toronto data

In [2]:
from bs4 import BeautifulSoup
import lxml
import requests
toronto_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print('Toronto data taken into account')


Toronto data taken into account


In [3]:
soup = BeautifulSoup(toronto_url.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

#Find all items and skip first one:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

# skip all meta symbols and line feeds between rows:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

# make dataframe, expand rows and drop the old one:
toronto_df=pd.DataFrame(rows)
toronto_df[headers] = toronto_df[0].str.split("</td>\n<td>", n = 2, expand = True) 
toronto_df.drop(columns=[0],inplace=True)

In [4]:
# skip not assigned boroughs:
toronto_df = toronto_df.drop(toronto_df[(toronto_df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
toronto_df.Neighbourhood.replace("Not assigned", toronto_df.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
toronto_df.Neighbourhood.fillna(toronto_df.Borough, inplace=True)
# drop duplicate rows:
toronto_df=toronto_df.drop_duplicates()

# extract titles from columns
toronto_df.update(
    toronto_df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

toronto_df.update(
    toronto_df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from Neighbourhood:
toronto_df.update(
    toronto_df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
toronto_df.update(
    toronto_df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':toronto_df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(toronto_df['Borough'].loc[toronto_df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(toronto_df['Neighbourhood'].loc[toronto_df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park


In [98]:
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Queen's Park,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### Importing NewYork Data

In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('New York Data downloaded!')

New York Data downloaded!


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### map of Toronto with neighborhoods superimposed on top

In [13]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

####  map of New York with neighborhoods superimposed on top

In [15]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [16]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

####  a brief summary of unique Borough we have from both dataframe and total rows.

In [17]:
print('The data set from New York has rows : ',format(neighborhoods.count().unique()))
print('The data set from Toronto has rows : ',format(toronto_data.count().unique()))

print('New York has boroughs: ',format(len(neighborhoods['Borough'].unique())))
print('Toronto has boroughs: ',format(len(toronto_data['Borough'].unique())))

The data set from New York has rows :  [306]
The data set from Toronto has rows :  [103]
New York has boroughs:  5
Toronto has boroughs:  12


#### Foursquare API initialisation

In [18]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: TI4KLLRUWUD3J544X5L1JHQ4YEDAWGKV5JJNBWQXAYH0ZENG
CLIENT_SECRET:KGOWK0Z0NN5OBR43KZDGUVXRRT4JEOWIYWHKHFMFUN4TFPLF


In [19]:
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins

#### Geographical Coordinates dataset

#### New York Geographical dataset

In [20]:
!wget -q -O 'newyork_data_Geo.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [21]:
with open('newyork_data_Geo.json') as json_data:
    newyork_data1 = json.load(json_data)

In [22]:
ny_data = newyork_data1['features']

In [23]:
ny_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [24]:
# define the dataframe columns
column_names1 = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny = pd.DataFrame(columns=column_names1)

In [25]:
for data in ny_data:
    borough = ny_name = data['properties']['borough'] 
    ny_name = data['properties']['name']
        
    ny_latlon = data['geometry']['coordinates']
    ny_lat = ny_latlon[1]
    ny_lon = ny_latlon[0]
    
    ny = ny.append({'Borough': borough,
                                          'Neighborhood': ny_name,
                                          'Latitude': ny_lat,
                                          'Longitude': ny_lon}, ignore_index=True)
    ny.head()

In [26]:
ny.to_csv('BON1_NYC_GEO.csv',index=False)

In [27]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [28]:
# create map of NewYork using latitude and longitude values
map_NewYork1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NewYork1)  
    
map_NewYork1

#### Toronto Geographical dataset

#### New York Markets

In [29]:
!conda install -c conda-forge wordcloud==1.4.1 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wordcloud==1.4.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wordcloud-1.4.1            |           py36_0         324 KB  conda-forge

The following NEW packages will be INSTALLED:

    wordcloud: 1.4.1-py36_0 conda-forge


wordcloud-1.4.1      | 324 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [31]:
# import package and its set of stopwords
!conda  install -c geopandas
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
mpl.style.use('ggplot') # optional: for ggplot-like style

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker

# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y
import seaborn as sns

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0


print ('Wordcloud is installed and imported!')


CondaValueError: too few arguments, must supply command line package specs or --file

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.27 |                0         132 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  anaconda
    seaborn-0.9.0              |     pyh91ea838_1         164 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following packages will be UPDATED:

    certifi:         2019.11.28-py36_0     conda-forge --> 2019.11.28-py36_0  anaconda
    openssl:         1.1.1d-h516909a_0     conda-forge --> 1.1.1-h7b644

In [47]:
#!pip install  psycopg2-binary
!pip install adal


     |████████████████████████████████| 61kB 14.9MB/s eta 0:00:01


In [60]:
import getopt
import sys
import json
import re
from adal import AuthenticationContext
from project_lib import project


In [61]:
# The code was removed by Watson Studio for sharing.


This data is extracted from the wikipedia Page - https://en.wikipedia.org/wiki/Cuisine_of_New_York_City

In [71]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Unnamed: 1
0,Data Dictionary - Dataset Information,NaN
1,NaN,NaN
2,General,NaN
3,Dataset Name,Farmers Markets and Food Boxes
4,Agency Name,DOHMH


In [106]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


#### Toronto Cuisine

In [80]:
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [99]:
toronto_data = toronto_data.groupby(['Postcode', 'Borough']).agg(','.join)

In [100]:
toronto_data.head(20)

,,Neighborhood
Postcode,Borough,
M1B,"Scarborough, Toronto","Rouge, Malvern"
M1C,"Scarborough, Toronto","Highland Creek , Port Union, Rouge Hill"
M1E,"Scarborough, Toronto","West Hill, Guildwood, Morningside"
M1G,"Scarborough, Toronto",Woburn
M1H,"Scarborough, Toronto",Cedarbrae
M1J,"Scarborough, Toronto",Scarborough Village
M1K,"Scarborough, Toronto","East Birchmount Park, Kennedy Park, Ionview"
M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Oakridge"
M1M,"Scarborough, Toronto","Scarborough Village West, Cliffcrest, Cliffside"


In [107]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [108]:
# The code was removed by Watson Studio for sharing.

In [110]:
print('Crawling different neighborhoods inside "Scarborough"')
Toronto_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postcode']),
                                                   list(scarborough_data['Neighborhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Toronto"


#### New York Cuisine

This data is extracted from the wikipedia Page - https://en.wikipedia.org/wiki/Cuisine_of_New_York_City

In [113]:
from PIL import Image
# Fetch the file
my_file = project.get_file("BON3_NYC_CUISINE.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
import pandas as pd
NYC_CUISINE=pd.read_csv(my_file)
NYC_CUISINE.drop(NYC_CUISINE.columns[[3,4,5,6,7]], axis=1,inplace=True) 
NYC_CUISINE.head()

AttributeError: module 'project_lib.project' has no attribute 'get_file'